In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver

In [16]:
class MessagesState(MessagesState):
    user_feedback: str

In [18]:
def step_1(state: MessagesState) -> None:
    print("---Step 1---")
    pass


def human_feedback(state: MessagesState) -> None:
    print("---human_feedback---")
    feedback = interrupt("Please provide feedback:")
    state["user_feedback"] = feedback
    return state


def step_3(state: MessagesState) -> None:
    print("---Step 3---")
    pass

In [19]:
builder = StateGraph(MessagesState)
builder.add_node("step_1", step_1)
builder.add_node("human_feedback", human_feedback)
builder.add_node("step_3", step_3)
builder.add_edge(START, "step_1")
builder.add_edge("step_1", "human_feedback")
builder.add_edge("human_feedback", "step_3")
builder.add_edge("step_3", END)

# Set up memory
memory = MemorySaver()

# Add
graph = builder.compile(checkpointer=memory)

In [21]:
initial_input = {"messages": "hello world"}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")

---Step 1---
{'step_1': None}


---human_feedback---
{'__interrupt__': (Interrupt(value='Please provide feedback:', resumable=True, ns=['human_feedback:18a9e874-2662-d67e-e314-d8e18d362476'], when='during'),)}




In [22]:
for event in graph.stream(
    Command(resume="go to step 3!"),
    thread,
    stream_mode="updates",
):
    print(event)
    print("\n")

---human_feedback---
{'human_feedback': {'messages': [HumanMessage(content='hello world', additional_kwargs={}, response_metadata={}, id='2e8a9249-b2f5-408a-bfc1-1973ea4a9853'), HumanMessage(content='hello world', additional_kwargs={}, response_metadata={}, id='1aed3892-1b7d-42ff-b29e-5a9add3a123a')], 'user_feedback': 'go to step 3!'}}


---Step 3---
{'step_3': None}


